# N-body problem

In [ ]:
import mylib.integration as integration
from mylib.model import three_body_model

import numpy as np

from scipy.integrate import solve_ivp

from bokeh.io import output_notebook
from bokeh.plotting import show, figure
from bokeh.palettes import Category20

output_notebook(hide_banner=True)

In [ ]:
def plot_sol():

    m1 = 9.54786104043e-4
    m2 = 2.85583733151e-4

    qini = np.zeros(9)
    qini[0] =  0.
    qini[1] =  0.
    qini[2] =  0.
    qini[3] = -3.5023653
    qini[4] = -3.8169847
    qini[5] = -1.5507963
    qini[6] =  9.0755314
    qini[7] = -3.0458353
    qini[8] = -1.6483708

    pini = np.zeros(9)
    pini[0] =  0.
    pini[1] =  0.
    pini[2] =  0.
    pini[3] =  0.00565429*m1
    pini[4] = -0.00412490*m1
    pini[5] = -0.00190589*m1
    pini[6] =  0.00168318*m2
    pini[7] =  0.00483525*m2
    pini[8] =  0.00192462*m2

    yini = np.concatenate((qini, pini))

    tini = 0.
    tend = 12500.
    nt = 1501

    tbm = three_body_model()
    H_p = tbm.H_p
    H_q = tbm.H_q
    fcn = tbm.fcn

    fig_xy = figure(plot_height=500, plot_width=950, title="xy (Click on legend on hide corresponding graph)")
    fig_ham = figure(x_range=(tini, tend), plot_height=500, plot_width=950, title="Hamiltonian H(t)")
    t = np.linspace(tini, tend, nt)
    
    # forward Euler integration
    sol_fe = integration.forward_euler(tini, tend, nt, yini, fcn)
    yfe = sol_fe.y
    ham_fe = tbm.hamiltonian(sol_fe.y)
    fig_xy.x(yfe[3]-yfe[0], yfe[4]-yfe[1], color=Category20[20][0], legend_label="forward Euler")
    fig_xy.x(yfe[6]-yfe[0], yfe[7]-yfe[1], color=Category20[20][1], legend_label="forward Euler")
    fig_ham.x(sol_fe.t, ham_fe, color=Category20[20][0], legend_label="forward Euler")

    # backward Euler integration
    sol_be = integration.backward_euler(tini, tend, nt, yini, fcn)
    ybe = sol_be.y
    ham_be = tbm.hamiltonian(sol_be.y)
    fig_xy.x(ybe[3]-ybe[0], ybe[4]-ybe[1], color=Category20[20][8], legend_label="backward Euler")
    fig_xy.x(ybe[6]-ybe[0], ybe[7]-ybe[1], color=Category20[20][9], legend_label="backward Euler")
    fig_ham.x(sol_be.t, ham_be, color=Category20[20][8], legend_label="backward Euler")

    # symplectic Euler 
    sol_se = integration.symplectic_euler(tini, tend, nt, yini, H_p, H_q)
    yse = sol_se.y
    ham_se = tbm.hamiltonian(sol_se.y)
    fig_xy.x(yse[3]-yse[0], yse[4]-yse[1], color=Category20[20][2], legend_label="symplectic Euler")
    fig_xy.x(yse[6]-yse[0], yse[7]-yse[1], color=Category20[20][3], legend_label="symplectic Euler")
    fig_ham.x(sol_se.t, ham_se, color=Category20[20][2], legend_label="symplectic Euler")

    # Stormer-Verlet
    sol_sv = integration.stormer_verlet(tini, tend, nt, yini, H_p, H_q)
    ysv = sol_sv.y
    ham_sv = tbm.hamiltonian(sol_sv.y)
    fig_xy.x(ysv[3]-ysv[0], ysv[4]-ysv[1], color=Category20[20][4], legend_label="Stormer-Verlet")
    fig_xy.x(ysv[6]-ysv[0], ysv[7]-ysv[1], color=Category20[20][5], legend_label="Stormer-Verlet")
    fig_ham.x(sol_sv.t, ham_sv, color=Category20[20][4], legend_label="Stormer-Verlet")

    # optimized order 8 step 15 integration
    sol_o815 = integration.optimized_815(tini, tend, nt, yini, H_p, H_q)
    yo815 = sol_o815.y
    ham_o815 = tbm.hamiltonian(sol_o815.y)
    fig_xy.x(yo815[3]-yo815[0], yo815[4]-yo815[1], color=Category20[20][6], legend_label="optimized order 8 step 15")
    fig_xy.x(yo815[6]-yo815[0], yo815[7]-yo815[1], color=Category20[20][7], legend_label="optimized order 8 step 15")
    fig_ham.x(sol_o815.t, ham_o815, color=Category20[20][6], legend_label="optimized order 8 step 15")
    
    # RK45
    sol_rk45 = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-6, atol=1.e-6)
    yrk45 = sol_rk45.y
    ham_rk45 = tbm.hamiltonian(sol_rk45.y)
    fig_xy.x(yrk45[3]-yrk45[0], yrk45[4]-yrk45[1], color=Category20[20][12], legend_label="rk45")
    fig_xy.x(yrk45[6]-yrk45[0], yrk45[7]-yrk45[1], color=Category20[20][13], legend_label="rk45")
    fig_ham.x(sol_rk45.t, ham_rk45, color=Category20[20][12], legend_label="rk45")
    
    # dopri853
    sol_dopri853 = solve_ivp(fcn, (tini, tend), yini, method="DOP853", rtol=1.e-6, atol=1.e-6)
    ydopri853 = sol_dopri853.y
    ham_dopri853 = tbm.hamiltonian(sol_dopri853.y)
    fig_xy.x(ydopri853[3]-ydopri853[0], ydopri853[4]-ydopri853[1], color=Category20[20][10], legend_label="dopri853")
    fig_xy.x(ydopri853[6]-ydopri853[0], ydopri853[7]-ydopri853[1], color=Category20[20][11], legend_label="dopri853")
    fig_ham.x(sol_dopri853.t, ham_dopri853, color=Category20[20][10], legend_label="dopri853")

    fig_xy.legend.background_fill_alpha = 0.75
    fig_xy.legend.click_policy="hide"
    show(fig_xy)
    
    fig_ham.legend.background_fill_alpha = 0.75
    fig_ham.legend.click_policy="hide"
    show(fig_ham)
    
plot_sol()

**La cellule suivante met longtemps à s'exécuter, c'est normal**

In [ ]:
def plot_sol_longtime():

    m1 = 9.54786104043e-4
    m2 = 2.85583733151e-4

    qini = np.zeros(9)
    qini[0] =  0.
    qini[1] =  0.
    qini[2] =  0.
    qini[3] = -3.5023653
    qini[4] = -3.8169847
    qini[5] = -1.5507963
    qini[6] =  9.0755314
    qini[7] = -3.0458353
    qini[8] = -1.6483708

    pini = np.zeros(9)
    pini[0] =  0.
    pini[1] =  0.
    pini[2] =  0.
    pini[3] =  0.00565429*m1
    pini[4] = -0.00412490*m1
    pini[5] = -0.00190589*m1
    pini[6] =  0.00168318*m2
    pini[7] =  0.00483525*m2
    pini[8] =  0.00192462*m2

    yini = np.concatenate((qini, pini))

    tini = 0.
    tend = 9500000.
    nt = 30001

    tbm = three_body_model()
    H_p = tbm.H_p
    H_q = tbm.H_q
    fcn = tbm.fcn

    fig_xy = figure(plot_height=500, plot_width=950, title="xy (Click on legend on hide corresponding graph)")
    fig_ham = figure(x_range=(tini, tend), plot_height=500, plot_width=950, title="Hamiltonian H(t)")
    t = np.linspace(tini, tend, nt)
    
    # Stormer-Verlet
    sol_sv = integration.stormer_verlet(tini, tend, nt, yini, H_p, H_q)
    ysv = sol_sv.y
    ham_sv = tbm.hamiltonian(sol_sv.y)
    fig_xy.x(ysv[3][::10]-ysv[0][::10], ysv[4][::10]-ysv[1][::10], color=Category20[20][4], legend_label="Stormer-Verlet")
    fig_xy.x(ysv[6][::10]-ysv[0][::10], ysv[7][::10]-ysv[1][::10], color=Category20[20][5], legend_label="Stormer-Verlet")
    fig_ham.x(sol_sv.t[::10], ham_sv[::10], color=Category20[20][4], legend_label="Stormer-Verlet")
    
    # optimized order 8 step 15 integration
    sol_o815 = integration.optimized_815(tini, tend, nt, yini, H_p, H_q)
    yo815 = sol_o815.y
    ham_o815 = tbm.hamiltonian(sol_o815.y)
    fig_xy.x(yo815[3][::10]-yo815[0][::10], yo815[4][::10]-yo815[1][::10], color=Category20[20][6], legend_label="optimized order 8 step 15")
    fig_xy.x(yo815[6][::10]-yo815[0][::10], yo815[7][::10]-yo815[1][::10], color=Category20[20][7], legend_label="optimized order 8 step 15")
    fig_ham.x(sol_o815.t[::10], ham_o815[::10], color=Category20[20][6], legend_label="optimized order 8 step 15")

    # RK45
    sol_rk45 = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-6, atol=1.e-6)
    yrk45 = sol_rk45.y
    ham_rk45 = tbm.hamiltonian(sol_rk45.y)
    fig_xy.x(yrk45[3][::10]-yrk45[0][::10], yrk45[4][::10]-yrk45[1][::10], color=Category20[20][12], legend_label="rk45")
    fig_xy.x(yrk45[6][::10]-yrk45[0][::10], yrk45[7][::10]-yrk45[1][::10], color=Category20[20][13], legend_label="rk45")
    fig_ham.x(sol_rk45.t[::10], ham_rk45[::10], color=Category20[20][12], legend_label="rk45")
    
    # dopri853
    sol_dopri853 = solve_ivp(fcn, (tini, tend), yini, method="DOP853", rtol=1.e-6, atol=1.e-6)
    ydopri853 = sol_dopri853.y
    ham_dopri853 = tbm.hamiltonian(sol_dopri853.y)
    fig_xy.x(ydopri853[3][::10]-ydopri853[0][::10], ydopri853[4][::10]-ydopri853[1][::10], color=Category20[20][10], legend_label="dopri853")
    fig_xy.x(ydopri853[6][::10]-ydopri853[0][::10], ydopri853[7][::10]-ydopri853[1][::10], color=Category20[20][11], legend_label="dopri853")
    fig_ham.x(sol_dopri853.t[::10], ham_dopri853[::10], color=Category20[20][10], legend_label="dopri853")

    fig_xy.legend.background_fill_alpha = 0.75
    fig_xy.legend.click_policy="hide"
    show(fig_xy)
    
    fig_ham.legend.background_fill_alpha = 0.75
    fig_ham.legend.click_policy="hide"
    show(fig_ham)
    
plot_sol_longtime()